In [ ]:
import datetime
import pandas as pd
import numpy as np
import os
import seaborn as sns
import re
import matplotlib.pyplot as plt
import warnings
import zipfile 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

warnings.filterwarnings("always")
warnings.filterwarnings("ignore")
sns.set(style="darkgrid")
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示

%matplotlib inline

In [ ]:
df = pd.read_csv("../input/pmsm_temperature_data.csv")

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
cor = df.corr()
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(cor, xticklabels=cor.columns, yticklabels=cor.columns, annot=True, ax=ax);

In [ ]:
extra_feats = {
     "i_s": lambda x: np.sqrt(x["i_d"]**2 + x["i_q"]**2),  # Current vector norm
     "u_s": lambda x: np.sqrt(x["u_d"]**2 + x["u_q"]**2),  # Voltage vector norm
     "S_el": lambda x: x["i_s"]*x["u_s"],                  # Apparent power
     "P_el": lambda x: x["i_d"] * x["u_d"] + x["i_q"] *x["u_q"],  # Effective power
     "i_s_x_w": lambda x: x["i_s"]*x["motor_speed"],
     "S_x_w": lambda x: x["S_el"]*x["motor_speed"],
}
df = df.assign(**extra_feats)

In [ ]:
df.hist(figsize = (20,20))
plt.show()

* ****relate between i_q and torque is 1，only need keep one****

In [ ]:
df_clean = df.drop(["profile_id", "i_q"],axis=1)
indexs = df_clean.index
columns = df_clean.columns

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df_clean)
df_clean = pd.DataFrame(scaler.transform(df_clean), index=indexs, columns=columns)
df_clean.head()

In [ ]:
pl_data = df_clean.copy()
pl_data.index = pl_data["pm"]
pl_data = pl_data.drop(columns=["pm"], axis=1)

In [ ]:
X = df_clean.drop(["pm"],axis=1)
y = df_clean["pm"]

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

* **LinearRegression**

In [ ]:
LR_model = LinearRegression()
LR_model.fit(X_train, y_train)

print("b0: ", LR_model.intercept_)
print("b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11 : ", LR_model.coef_)

In [ ]:
def access(y_pred, y_true):
    print("r2_score : ", r2_score(y_true, y_pred))
    print("mean_squared_error : ", mean_squared_error(y_true, y_pred))
          

In [ ]:
print("train")
access(LR_model.predict(X_train), y_train)
print("\ntest")
access(LR_model.predict(X_test), y_test)

* **Keras RNN model**

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

In [ ]:
input_dim = X_train.shape[1]
hidden_1 = 64
hidden_2 = 64
hidden_3 = 64
output_dim = 1
batch_size = 100
epochs = 10

X = Input(shape=[input_dim,])
h = Dense(hidden_1, activation='relu')(X)
h = Dense(hidden_2, activation='relu')(h)
h = Dense(hidden_3, activation='relu')(h)
Y = Dense(output_dim, activation='sigmoid')(h)

keras_model = Model(X, Y)
keras_model.compile(loss='mean_squared_error', optimizer='adam')
history = keras_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=False, validation_split=0.2)

In [ ]:
y_pred = keras_model.predict(X_test)
print('MSE Train:', keras_model.evaluate(X_train, y_train, batch_size=batch_size))
print('MSE Test:', keras_model.evaluate(X_test, y_test, batch_size=batch_size))

In [ ]:
pd.DataFrame(history.history)[["loss", "val_loss"]].plot(figsize=(16, 6));